# Syntax preprocessing

This tutorial describes the tools that are needed to convert the morphologically analysed Estonian text into the format in which it can be parsed with **VISL-CG3 parser**. VISL CG3 is a rule-based syntactic parser that has thousands of Estonian-specific handcrafted rules for tagging syntactic functions and dependencies. However, the parser needs a little more information than is given out by the standard morphological analyser (e.g. pronoun types, verb subcategorization, etc.), and its input needs to be in a different format. Therefore, several taggers, rewriters, and an exporter that are described in the rest of the tutorial are needed.

## Syntax preprocessing taggers and rewriters


|tagger|rewriters|source attributes|target attributes|values|
|------|---------|-----------------|-----------------|------|
|&nbsp;| PunctuatinonTypeRewriter | partofspeech, root|punctuation_type|```None```, 'Fst', 'Com', 'Col', ... |
|&nbsp;| MorphToSyntaxMorphRewriter|partofspeech, form|partofspeech, form||
|PronounTypeTagger|PronounTypeRewriter|root, ending, clitic, partofspeech|pronoun_type| ```None```, ('det',), ('pers ps3',), ('pos', 'det', 'refl'), ... |
|&nbsp;|RemoveDuplicateAnalysesRewriter|root, ending, clitic, partofspeech, form|&nbsp;|&nbsp;|
|&nbsp;|RemoveAdpositionAnalysesRewriter|partofspeech, form|&nbsp;|&nbsp;|
|&nbsp;|LetterCaseRewriter|word_text|cap|```None```, 'cap'|
|FiniteFormTagger|MorphToSyntaxMorphRewriter, FiniteFormRewriter|partofspeech, form|fin|```None```, ```True```, ```False```|
|VerbExtensionSuffixTagger|VerbExtensionSuffixRewriter|root|verb_extension_suffix|```None```,'tud','nud','mine','nu','tu','v','tav','mata','ja'|
|SubcatTagger|MorphToSyntaxMorphRewriter, SubcatRewriter|root, partofspeech, form|subcat|```None```, 'Intr', 'Part', 'gen', ...|
|MorphExtendedTagger|PunctuatinonTypeRewriter, MorphToSyntaxMorphRewriter, PronounTypeRewriter, RemoveDuplicateAnalysesRewriter, RemoveAdpositionAnalysesRewriter, LetterCaseRewriter, FiniteFormRewriter, VerbExtensionSuffixRewriter, SubcatRewriter|root, ending, clitic, partofspeech, form|partofspeech, form, punctuation_type, pronoun_type, cap, fin, verb_extension_suffix, subcat ||

In [1]:
from estnltk import Text
from estnltk.core import abs_path

## PronounTypeRetagger

Create an instance of `PronounTypeRetagger`.

In [2]:
from estnltk.taggers import PronounTypeRetagger

retagger = PronounTypeRetagger('morph_analysis')
retagger

PronounTypeRetagger(pronoun_type_rewriter=<estnltk.rewriting.syntax_preprocessing.pronoun_type_rewriter.PronounTypeRewriter object at 0x7fb739dd8f28>)

Run retagger and observe that the `pronoun_type` attribute is added to the `morph_analysis` layer.

In [3]:
text = Text('Kumb, sina või mina?').tag_layer()

retagger.retag(text)
text.morph_analysis

Layer(name='morph_analysis', attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'pronoun_type'), spans=SL[Span('Kumb', [{'lemma': 'kumb', 'root': 'kumb', 'root_tokens': ('kumb',), 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'P', 'pronoun_type': ('rel',)}]),
Span(',', [{'lemma': ',', 'root': ',', 'root_tokens': (',',), 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z', 'pronoun_type': None}]),
Span('sina', [{'lemma': 'sina', 'root': 'sina', 'root_tokens': ('sina',), 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'P', 'pronoun_type': ('ps2',)}]),
Span('või', [{'lemma': 'või', 'root': 'või', 'root_tokens': ('või',), 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'J', 'pronoun_type': None}]),
Span('mina', [{'lemma': 'mina', 'root': 'mina', 'root_tokens': ('mina',), 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S', 'pronoun_type': None}]),
Span('?', [{'lemma': '?', 'root': '?', 'root_tokens': ('?',), 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z', 'pronoun_type': None}])])

## FiniteFormRetagger

In [4]:
from estnltk.taggers import FiniteFormTagger

fsToSyntFulesFile = abs_path('rewriting/syntax_preprocessing/rules_files/tmorftrtabel.txt')

tagger = FiniteFormTagger(fs_to_synt_rules_file=fsToSyntFulesFile)
tagger

name,layer,attributes,depends_on
FiniteFormTagger,finite_form,"(lemma, root, root_tokens, ending, clitic, form, partofspeech, fin)",[morph_analysis]


In [5]:
from estnltk.taggers.syntax_preprocessing.finite_form_tagger import FiniteFormRetagger

fsToSyntFulesFile = abs_path('rewriting/syntax_preprocessing/rules_files/tmorftrtabel.txt')

retagger = FiniteFormRetagger('morph_analysis', fs_to_synt_rules_file=fsToSyntFulesFile)
retagger

FiniteFormRetagger(function=<function FiniteFormRetagger.__init__.<locals>.function at 0x7fb73a7a5620>)

In [6]:
text = Text('laulma hüpelnud tantsija').tag_layer()
text.morph_analysis

retagger.retag(text)

text.morph_analysis

Layer(name='morph_analysis', attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'fin'), spans=SL[Span('laulma', [{'lemma': 'laulma', 'root': 'laul', 'root_tokens': ('laul',), 'ending': 'ma', 'clitic': '', 'form': 'mod sup ps ill', 'partofspeech': 'V', 'fin': False}, {'lemma': 'laulma', 'root': 'laul', 'root_tokens': ('laul',), 'ending': 'ma', 'clitic': '', 'form': 'aux sup ps ill', 'partofspeech': 'V', 'fin': False}, {'lemma': 'laulma', 'root': 'laul', 'root_tokens': ('laul',), 'ending': 'ma', 'clitic': '', 'form': 'main sup ps ill', 'partofspeech': 'V', 'fin': False}]),
Span('hüpelnud', [{'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ('hüpelnud',), 'ending': '0', 'clitic': '', 'form': 'pos', 'partofspeech': 'A', 'fin': None}, {'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ('hüpelnud',), 'ending': '0', 'clitic': '', 'form': 'pos sg nom', 'partofspeech': 'A', 'fin': None}, {'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ('hüpelnud',), 'ending': 'd', 'clitic': '', 'form': 'pos pl nom', 'partofspeech': 'A', 'fin': None}, {'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ('hüple',), 'ending': 'nud', 'clitic': '', 'form': 'mod indic impf ps neg', 'partofspeech': 'V', 'fin': True}, {'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ('hüple',), 'ending': 'nud', 'clitic': '', 'form': 'mod partic past ps', 'partofspeech': 'V', 'fin': False}, {'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ('hüple',), 'ending': 'nud', 'clitic': '', 'form': 'aux indic impf ps neg', 'partofspeech': 'V', 'fin': True}, {'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ('hüple',), 'ending': 'nud', 'clitic': '', 'form': 'aux partic past ps', 'partofspeech': 'V', 'fin': False}, {'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ('hüple',), 'ending': 'nud', 'clitic': '', 'form': 'main indic impf ps neg', 'partofspeech': 'V', 'fin': True}, {'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ('hüple',), 'ending': 'nud', 'clitic': '', 'form': 'main partic past ps', 'partofspeech': 'V', 'fin': False}]),
Span('tantsija', [{'lemma': 'tantsija', 'root': 'tantsija', 'root_tokens': ('tantsija',), 'ending': '0', 'clitic': '', 'form': 'com sg nom', 'partofspeech': 'S', 'fin': None}])])

## VerbExtensionSuffixRetagger

In [7]:
from estnltk.taggers import VerbExtensionSuffixRetagger

retagger = VerbExtensionSuffixRetagger('morph_analysis')
retagger

VerbExtensionSuffixRetagger(function=<function VerbExtensionSuffixRetagger.__init__.<locals>.function at 0x7fb739dab1e0>)

In [8]:
text = Text('Laulev hüpelnud tantsija').tag_layer()
retagger.retag(text)
text.morph_analysis

Layer(name='morph_analysis', attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'verb_extension_suffix'), spans=SL[Span('Laulev', [{'lemma': 'laulev', 'root': 'laulev', 'root_tokens': ('laulev',), 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'A', 'verb_extension_suffix': ()}]),
Span('hüpelnud', [{'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ('hüpelnud',), 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'A', 'verb_extension_suffix': ('nud',)}, {'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ('hüpelnud',), 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'A', 'verb_extension_suffix': ('nud',)}, {'lemma': 'hüpelnud', 'root': 'hüpel=nud', 'root_tokens': ('hüpelnud',), 'ending': 'd', 'clitic': '', 'form': 'pl n', 'partofspeech': 'A', 'verb_extension_suffix': ('nud',)}, {'lemma': 'hüplema', 'root': 'hüple', 'root_tokens': ('hüple',), 'ending': 'nud', 'clitic': '', 'form': 'nud', 'partofspeech': 'V', 'verb_extension_suffix': ()}]),
Span('tantsija', [{'lemma': 'tantsija', 'root': 'tantsija', 'root_tokens': ('tantsija',), 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S', 'verb_extension_suffix': ()}])])

## SubcatRetagger

In [9]:
from os.path import relpath
from estnltk.taggers import SubcatRetagger

fsToSyntFulesFile = relpath(abs_path('rewriting/syntax_preprocessing/rules_files/tmorftrtabel.txt'))
subcatFile = relpath(abs_path('rewriting/syntax_preprocessing/rules_files/abileksikon06utf.lx'))

tagger = SubcatRetagger('morph_analysis',
                        fs_to_synt_rules_file=fsToSyntFulesFile, 
                        subcat_rules_file=subcatFile)
tagger

SubcatRetagger(function=<function SubcatRetagger.__init__.<locals>.function at 0x7fb739ef3d08>)

In [10]:
text = Text('Järel juurduma').tag_layer()
tagger.retag(text)
text.morph_analysis

Layer(name='morph_analysis', attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'subcat'), spans=SL[Span('Järel', [{'lemma': 'järel', 'root': 'järel', 'root_tokens': ('järel',), 'ending': '0', 'clitic': '', 'form': 'post', 'partofspeech': 'K', 'subcat': ('gen',)}, {'lemma': 'järel', 'root': 'järel', 'root_tokens': ('järel',), 'ending': '0', 'clitic': '', 'form': 'pre', 'partofspeech': 'K', 'subcat': None}]),
Span('juurduma', [{'lemma': 'juurduma', 'root': 'juurdu', 'root_tokens': ('juurdu',), 'ending': 'ma', 'clitic': '', 'form': 'mod sup ps ill', 'partofspeech': 'V', 'subcat': ('Intr',)}, {'lemma': 'juurduma', 'root': 'juurdu', 'root_tokens': ('juurdu',), 'ending': 'ma', 'clitic': '', 'form': 'aux sup ps ill', 'partofspeech': 'V', 'subcat': ('Intr',)}, {'lemma': 'juurduma', 'root': 'juurdu', 'root_tokens': ('juurdu',), 'ending': 'ma', 'clitic': '', 'form': 'main sup ps ill', 'partofspeech': 'V', 'subcat': ('Intr',)}])])

## MorphExtendedTagger

In [11]:
from estnltk.taggers import MorphExtendedTagger

fsToSyntFulesFile = relpath(abs_path('rewriting/syntax_preprocessing/rules_files/tmorftrtabel.txt'))
subcat_file = relpath(abs_path('rewriting/syntax_preprocessing/rules_files/abileksikon06utf.lx'))

tagger = MorphExtendedTagger(fs_to_synt_rules_file=fsToSyntFulesFile, 
                             allow_to_remove_all=False, 
                             subcat_rules_file=subcat_file)
tagger

MorphExtendedTagger(input_layers=('morph_analysis',), output_layer=morph_extended, output_attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'pu ..., type: <class 'tuple'>, length: 13, fs_to_synt_rules_file=../../estnltk/rewriting/syntax_preprocessing/rules_files/tmorftrtabel.txt, allow_to_remove_all=False, subcat_rules_file=../../estnltk/rewriting/syntax_preprocessing/rules_files/abileksikon06utf.lx, quick_morph_extended_rewriter=<estnltk.rewriting.syntax_preprocessing.morph_extended_rewriter.MorphExtendedRew ..., type: <class 'estnltk.rewriting.syntax_preprocessing.morph_extended_rewriter.MorphExtendedRewriter'>)

In [12]:
text = Text('Ta on rääkinud!').tag_layer()
tagger.tag(text)
text['morph_extended']

Layer(name='morph_extended', attributes=('lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech', 'punctuation_type', 'pronoun_type', 'letter_case', 'fin', 'verb_extension_suffix', 'subcat'), spans=SL[Span('Ta', [{'lemma': 'tema', 'root': 'tema', 'root_tokens': ('tema',), 'ending': '0', 'clitic': '', 'form': 'sg nom', 'partofspeech': 'P', 'punctuation_type': None, 'pronoun_type': ('ps3',), 'letter_case': 'cap', 'fin': None, 'verb_extension_suffix': (), 'subcat': None}]),
Span('on', [{'lemma': 'olema', 'root': 'ole', 'root_tokens': ('ole',), 'ending': '0', 'clitic': '', 'form': 'mod indic pres ps3 sg ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': (), 'subcat': ('Intr',)}, {'lemma': 'olema', 'root': 'ole', 'root_tokens': ('ole',), 'ending': '0', 'clitic': '', 'form': 'aux indic pres ps3 sg ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': (), 'subcat': ('Intr',)}, {'lemma': 'olema', 'root': 'ole', 'root_tokens': ('ole',), 'ending': '0', 'clitic': '', 'form': 'main indic pres ps3 sg ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': (), 'subcat': ('Intr',)}, {'lemma': 'olema', 'root': 'ole', 'root_tokens': ('ole',), 'ending': '0', 'clitic': '', 'form': 'mod indic pres ps3 pl ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': (), 'subcat': ('Intr',)}, {'lemma': 'olema', 'root': 'ole', 'root_tokens': ('ole',), 'ending': '0', 'clitic': '', 'form': 'aux indic pres ps3 pl ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': (), 'subcat': ('Intr',)}, {'lemma': 'olema', 'root': 'ole', 'root_tokens': ('ole',), 'ending': '0', 'clitic': '', 'form': 'main indic pres ps3 pl ps af', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': (), 'subcat': ('Intr',)}]),
Span('rääkinud', [{'lemma': 'rääkinud', 'root': 'rääki=nud', 'root_tokens': ('rääkinud',), 'ending': '0', 'clitic': '', 'form': 'pos', 'partofspeech': 'A', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': None, 'verb_extension_suffix': ('nud',), 'subcat': None}, {'lemma': 'rääkinud', 'root': 'rääki=nud', 'root_tokens': ('rääkinud',), 'ending': '0', 'clitic': '', 'form': 'pos sg nom', 'partofspeech': 'A', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': None, 'verb_extension_suffix': ('nud',), 'subcat': None}, {'lemma': 'rääkinud', 'root': 'rääki=nud', 'root_tokens': ('rääkinud',), 'ending': 'd', 'clitic': '', 'form': 'pos pl nom', 'partofspeech': 'A', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': None, 'verb_extension_suffix': ('nud',), 'subcat': None}, {'lemma': 'rääkima', 'root': 'rääki', 'root_tokens': ('rääki',), 'ending': 'nud', 'clitic': '', 'form': 'mod indic impf ps neg', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': (), 'subcat': ('Part-P', 'El')}, {'lemma': 'rääkima', 'root': 'rääki', 'root_tokens': ('rääki',), 'ending': 'nud', 'clitic': '', 'form': 'mod partic past ps', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': False, 'verb_extension_suffix': (), 'subcat': ('Part-P', 'El')}, {'lemma': 'rääkima', 'root': 'rääki', 'root_tokens': ('rääki',), 'ending': 'nud', 'clitic': '', 'form': 'aux indic impf ps neg', 'partofspeech': 'V', 'punctuation_type': None, 'pronoun_type': None, 'letter_case': None, 'fin': True, 'verb_extension_suffix': (), 'subcat': ('Part-P', 'El')}, {'lemma': 'rääkima', 'root': 'rääki', 'root_tokens': ('rääki',), 'ending': 'nud', 'clit

## CG3 exporter

In [13]:
from estnltk.converters.CG3_exporter import export_CG3
text = Text('Lähme! Ta on rääkinud.')
text.analyse('syntax_preprocessing')
export_CG3(text)

['"<s>"',
 '"<Lähme>"',
 '    "mine" Lme V mod indic pres ps1 pl ps af cap <FinV>',
 '    "mine" Lme V aux indic pres ps1 pl ps af cap <FinV>',
 '    "mine" Lme V main indic pres ps1 pl ps af cap <FinV>',
 '"<!>"',
 '    "!" Z Exc',
 '"</s>"',
 '"<s>"',
 '"<Ta>"',
 '    "tema" L0 P pers ps3 sg nom cap',
 '"<on>"',
 '    "ole" L0 V mod indic pres ps3 sg ps af <FinV> <Intr>',
 '    "ole" L0 V aux indic pres ps3 sg ps af <FinV> <Intr>',
 '    "ole" L0 V main indic pres ps3 sg ps af <FinV> <Intr>',
 '    "ole" L0 V mod indic pres ps3 pl ps af <FinV> <Intr>',
 '    "ole" L0 V aux indic pres ps3 pl ps af <FinV> <Intr>',
 '    "ole" L0 V main indic pres ps3 pl ps af <FinV> <Intr>',
 '"<rääkinud>"',
 '    "rääki=nud" L0 A pos partic <nud>',
 '    "rääki=nud" L0 A pos sg nom partic <nud>',
 '    "rääki=nud" Ld A pos pl nom partic <nud>',
 '    "rääki" Lnud V mod indic impf ps neg <FinV> <Part-P> <El>',
 '    "rääki" Lnud V mod partic past ps <Part-P> <El>',
 '    "rääki" Lnud V aux indic impf p